In [ ]:
import json

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from target_toolbox.aruco_marker import ARUCO_DICT_TYPE
from target_toolbox.aruco_sine_chart import generate_aruco_sine_chart_meta, extract_sine_and_bw_tiles, \
                                            estimate_comm_diff_from_bw_tile, estimate_mtf_from_sine_tile, \
                                            find_sine_corner_list, draw_sine_block_outline_and_mtf

## MTF estimation theoretical validation

In [ ]:
from target_toolbox.common import mm_to_pixels, pixels_to_mm, dpi_to_pp, pp_to_dpi, remove_bezel
from target_toolbox.sine_chart import draw_sine_tile

In [ ]:
def _constrast(arr):
    return (arr.max() - arr.min()) / (arr.max() + arr.min())

def gaussian_kernel_mtf_func(sigma):
    # sigma: gaussian kernel simga in meters
    return lambda lpmm: np.exp(-np.power(lpmm*1e3*2*np.pi * sigma, 2)/2)

In [ ]:
lpmm = 0.5
sine_oversample = 16
pp = 1 / (lpmm * 2 * sine_oversample)
dpi = dpi_to_pp(pp)

length = 100 # in mm
height = 20

# prepare tiles with different blury
sine_tile = draw_sine_tile(lpmm, length, height, dpi)
sine_tile = cv.cvtColor(sine_tile, cv.COLOR_BGR2GRAY)
k_list = (2, 4, 6, 8, 10, 12)
blur_tile_list = []
for k in k_list:
    blur_tile_list.append(cv.GaussianBlur(sine_tile, (k*6+1,k*6+1), k, k))
tile_list = [sine_tile] + blur_tile_list

# calculate theoretical MTF using Gaussian kernel
theo_mtf_list = []
for k in k_list:
    k_mm = k * pp
    theo_mtf_list.append(gaussian_kernel_mtf_func(k_mm*1e-3)(lpmm))
theo_mtf_list = [1.0] + theo_mtf_list

# calculate contrast based MTF
cont_mtf_list = []
for tile in tile_list:
    cont_mtf_list.append(_constrast(remove_bezel(tile, 0.2).astype(float)/255.0))

# calculate spectrum based MTF
spec_mtf_list = []
for tile in tile_list:
    spec_mtf_list.append(estimate_mtf_from_sine_tile(tile.astype(float)/255.0, lpmm, pp, bezel_ratio=0.2))

# demo
for a in range(len(tile_list)):
    plt.figure(figsize=(16, 4))
    plt.imshow(remove_bezel(tile_list[a], 0.2), vmin=0, vmax=255)
    plt.title('Theoretical: {:.3f}. Contrast: {:.3f}. Spectrum: {:.3f}'.format( \
              theo_mtf_list[a], cont_mtf_list[a], spec_mtf_list[a]))
    plt.show()

## Generate test board

In [ ]:
# aruco parameters
aruco_idx = 12
aruco_length = 100

# sine/bw paramters
lpmm_list = [1/8, 1/6, 1/4, 1/3, 1/2]
length = 100
height = 20
side_width = 20
dpi = 600

In [ ]:
# generate chart
total_pattern, meta_dict = generate_aruco_sine_chart_meta(aruco_idx, aruco_length, 
                                                          lpmm_list, length, height,
                                                          side_width, dpi)

plt.figure(figsize=(16, 12))
plt.imshow(total_pattern)
plt.show()

print(meta_dict)

In [ ]:
# save image and meta data
fn = 'aruco_sine_chart_{:d}'.format(meta_dict['aruco_idx'])
cv.imwrite(fn+'.png', cv.cvtColor(total_pattern, cv.COLOR_BGR2GRAY))
with open(fn+'.json', 'w') as fp:
    json.dump(meta_dict, fp, indent=4)

## Psudo-detection test

In [ ]:
# aruco detector parameters
aruco_parameter = cv.aruco.DetectorParameters()
aruco_dict = cv.aruco.getPredefinedDictionary(ARUCO_DICT_TYPE)
aruco_detector = cv.aruco.ArucoDetector(aruco_dict, aruco_parameter)

In [ ]:
# project to frame
test_board = 255 - total_pattern
frame_w, frame_h = 3840, 2160

rng = np.random.default_rng()
s = frame_w / test_board.shape[1] * (rng.random()*0.2+0.1)
r = rng.random()*np.pi*2
x, y = (0.3 + rng.random(2) * 0.4) * np.array([frame_w, frame_h])
k = rng.random() * 4 + 1

H_pre = np.array([[s*np.cos(r), -s*np.sin(r), x],
                  [s*np.sin(r),  s*np.cos(r), y]], dtype=np.float32)
warped_board = cv.warpAffine(test_board, H_pre, (frame_w, frame_h), flags=cv.INTER_AREA)
warped_board = cv.GaussianBlur(warped_board, (0,0), k)

frame = 255 - warped_board

plt.figure(figsize=(16, 9))
plt.imshow(frame)
plt.show()

In [ ]:
# detect aruco
cornerList, idList, rejectedImgPoints = aruco_detector.detectMarkers(frame)
aruco_corner_list = np.array(cornerList[0], dtype=np.float32).reshape(4,2)

# extract tiles
tile_list, pp_list, lpmm_list = extract_sine_and_bw_tiles(cv.cvtColor(frame, cv.COLOR_BGR2GRAY), 
                                                          aruco_corner_list, meta_dict)

# calculate black/white contrast
bw_tile = tile_list[-1].astype(float)/255.0
comm_mode, diff_mode = estimate_comm_diff_from_bw_tile(bw_tile)

# calculate spectrum and MTF
mtf_list = []
for tile, lpmm, pp in zip(tile_list[:-1], lpmm_list, pp_list):
    mtf = estimate_mtf_from_sine_tile(tile.astype(float)/255.0, lpmm, pp, comm_mode, diff_mode, bezel_ratio=0.15)
    mtf_list.append(mtf)

In [ ]:
# find and draw sine block outline
sine_corner_list = find_sine_corner_list(aruco_corner_list, meta_dict)
frame_disp = np.copy(frame)
frame_disp = draw_sine_block_outline_and_mtf(frame_disp, sine_corner_list, lpmm_list, mtf_list)

plt.figure(figsize=(16, 9))
plt.imshow(frame_disp)
plt.show()